#실행전 준비

데이터 출처: https://aihub.or.kr/aihubdata/data/view.do?currMenu=115&topMenu=100&dataSetSn=580

In [ ]:
# pip installs

!pip install -q -U transformers bitsandbytes protobuf
!pip install -q -U peft trl matplotlib wandb

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
from google.colab import files

src = list(files.upload().values())[0]
open('make_data2.py','wb').write(src)

#라이브러리 import

In [ ]:
import os
import re
import math
from tqdm import tqdm
from google.colab import userdata
from huggingface_hub import login
import torch
import transformers
from transformers import AutoModelForCausalLM, AutoModelForImageTextToText, AutoTokenizer, TrainingArguments, set_seed, BitsAndBytesConfig
from datasets import load_dataset, Dataset, DatasetDict
import wandb
from peft import LoraConfig
from trl import SFTTrainer, SFTConfig
from datetime import datetime
import matplotlib.pyplot as plt
import pickle

from make_data2 import Datasetup

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

AttributeError: 'MessageFactory' object has no attribute 'GetPrototype'

...

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

#데이터 로드

In [ ]:
with open("/content/drive/MyDrive/law_ai/new_pickle/train.pkl","rb") as fr:
    train_data = pickle.load(fr)

In [ ]:
with open("/content/drive/MyDrive/law_ai/new_pickle/test.pkl","rb") as fr:
    test_data = pickle.load(fr)

In [ ]:
with open("/content/drive/MyDrive/law_ai/new_pickle/val.pkl","rb") as fr:
    val_data = pickle.load(fr)

In [ ]:
train_data[1]

<채무부존재확인
2015나2030020>

In [ ]:
print(len(train_data))
print(len(test_data))
print(len(val_data))

7801
100
100


#모델 학습 준비

In [ ]:
#모델 설정
MODEL_ID = "google/gemma-3-4b-pt"
TOKENIZER_MODEL = "google/gemma-3-4b-it"
PROJECT_NAME = "law_ai5"
HF_USER = "dlddu123"

#모델 불러오는 라이브러리 선택
if MODEL_ID == "google/gemma-3-1b-pt":
    MODEL_CLASS = AutoModelForCausalLM
else:
    MODEL_CLASS = AutoModelForImageTextToText

#모델의 dtype 설정
if torch.cuda.get_device_capability()[0] >= 8:
    torch_dtype = torch.bfloat16
else:
    torch_dtype = torch.float16

#프로젝트 이름 설정
RUN_NAME =  f"{datetime.now():%Y-%m-%d_%H.%M.%S}"
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
HUB_MODEL_NAME = f"{HF_USER}/{PROJECT_RUN_NAME}"

#LoRA 하이퍼파라미터 설정
peft_config = LoraConfig(
    r = 32,
    lora_alpha  = 32,
    target_modules = ['k_proj', 'q_proj', 'v_proj', 'o_proj', "gate_proj", "down_proj", "up_proj"],
    lora_dropout  = 0.1,
    task_type="CAUSAL_LM"
)

#WANDB 설정
EPOCHS = 10
BATCH_SIZE = 4
GRADIENT_ACCUMULATION_STEPS = 4
LEARNING_RATE = 1e-4
LR_SCHEDULER_TYPE = 'cosine'
WARMUP_RATIO = 0.03
OPTIMIZER = "adamw_torch_fused"

#WANDB 설정
LOG_TO_WANDB = True

%matplotlib inline

In [ ]:
HUB_MODEL_NAME

'dlddu123/law_ai5-2025-12-04_21.54.03'

In [ ]:
# Log in to HuggingFace

hf_token = userdata.get('HF_TOKEN')
login(hf_token, add_to_git_credential=True)

#WANDB 준비 및 시작

In [ ]:
wandb_api_key = userdata.get('WANDB_API_KEY')
os.environ["WANDB_API_KEY"] = wandb_api_key#from google.colab import userdata으로 colab에 저장된 키 가져오기

#WANDB 프로젝트 설정
os.environ["WANDB_PROJECT"] = PROJECT_NAME
os.environ["WANDB_LOG_MODEL"] = "checkpoint" if LOG_TO_WANDB else "end"
os.environ["WANDB_WATCH"] = "gradients"

wandb.login()

/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)
wandb: Currently logged in as: colabear9806 to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


True

In [ ]:
#혹시나 wandb가 이미 실행되어있을경우 초기화
try:
    if wandb.run is not None:
        wandb.finish()
except Exception:
    pass


In [ ]:
wandb_api_key = userdata.get('WANDB_API_KEY')  # 주피터/코랩 비밀변수 등에서 가져오는 경우
if not wandb_api_key or not str(wandb_api_key).strip():
    raise ValueError("WANDB_API_KEY가 비어 있습니다. userdata 등에 키를 설정해주세요.")

In [ ]:
wandb.login(key=wandb_api_key, relogin=True)

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: W&B API key is configured. Use `wandb login --relogin` to force relogin


True

In [ ]:
#wandb 실행 시작
if LOG_TO_WANDB:
  wandb.init(project=PROJECT_NAME, name=RUN_NAME)

#모델 로드 설정

In [ ]:
# 양자화 설정
QUANT_4_BIT = True
if QUANT_4_BIT:
  quant_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
    bnb_4bit_quant_type="nf4"
  )
else:
  quant_config = BitsAndBytesConfig(
    load_in_8bit=True,
    bnb_8bit_compute_dtype=torch.bfloat16
  )

In [ ]:
# tokenizer와 model 로드
tokenizer = AutoTokenizer.from_pretrained(TOKENIZER_MODEL, trust_remote_code=True)

base_model = MODEL_CLASS.from_pretrained(#멀티모달은 AutoModelForImageTextToText로 불러오기 아니면 일반 text는 AutoModelForCausalLM
    MODEL_ID,
    quantization_config=quant_config,
    dtype = torch_dtype,
    attn_implementation = "sdpa",
    device_map="auto",
)

print(f"Memory footprint: {base_model.get_memory_footprint() / 1e6:.1f} MB")

tokenizer_config.json:   0%|          | 0.00/1.16M [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/4.69M [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/33.4M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/35.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/662 [00:00<?, ?B/s]

...

model-00001-of-00002.safetensors:   0%|          | 0.00/4.96G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.64G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/215 [00:00<?, ?B/s]

Memory footprint: 3170.6 MB


#학습에 사용할 데이터 추출

In [ ]:
train_data[0].prompt

{'messages': [{'role': 'system',
   'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.'},
  {'role': 'user',
   'content': '사건유형 민사 세부유형 민사 2심 원고 측의 주장 피고는 쓰레기매립으로 인해 토지 소유권을 침해하여 원고에게 손해를 발생시킨 것이므로, 토지 정화 및 복토 비용 상당액(153,463,420원)을 지급해야 합니다. 피고 측의 주장 원고는 토지를 주택으로 사용하고 있어, 현재 손해를 입지 않았습니다. 피고의 쓰레기매립지 사용으로 인해 원고에게 손해가 발생한 것은 아니며, 10년 동안 손해배상 채권이 소멸시효 완성되었으므로 원고는 피고에게 손해배상 청구할 권리가 있습니다. 기초사실 요약 피고는 1984년부터 하천의 쓰레기매립지였던 16,296㎡의 쓰레기매립지를 생활쓰레기로 매립하다가 1988년 사용을 종료했습니다. 그 이후, 000 대 363㎡의 토지에 매립된 쓰레기량에 대해 소유권 이전 등기(2010년 7월)를 완료했습니다. 피고는 이 토지를 주택으로 사용하면서, 굴착 과정에서 1.5m에서 4m 지점 사이의 쓰레기 907.5㎥의 양이 매립되어 있다는 사실을 인지하고 있습니다. 이는 상당한 양의 쓰레기가 지하에 매장된 상황입니다.'},
  {'role': 'assistant',
   'content': '관련 법조항 민법 제214조 민법 제209조 민법 제761조 결론 제1심 판결을 취소한다. 피고는 원고에게 000 (주소 2 생략) 대 363㎡ 토지 지하 907.5㎥의 쓰레기 매립물을 모두 제거하여 그 토지를 원상으로 복구하라. 소송총비용은 피고가 부담한다. 제2항은 가집행할 수 있다.'}]}

In [ ]:
train_list = []
val_list = []
test_list = []


for prompt in train_data:
      train_list.append(prompt.prompt)
for prompt in test_data:
      test_list.append(prompt.prompt)
for prompt in val_data:
      val_list.append(prompt.prompt)

In [ ]:
train_dataset = Dataset.from_list(train_list)
eval_dataset = Dataset.from_list(val_list)
test_dataset = Dataset.from_list(test_list)

In [ ]:
#관련 법조항까지 학습을 시키면 제대로된 결과가 나오지 않아서 제외시키기
def del_law_list(target):
  obj = str(target['messages'][2]['content'].split('결론')[1:]).strip()
  target['messages'][2]['content'] = obj
  return target

In [ ]:
test_dataset[1]

{'messages': [{'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.',
   'role': 'system'},
  {'content': '사건유형 민사 세부유형 민사 1심 원고 측의 주장 없음 피고 측의 주장 없음 기초사실 요약 1935년생 원고는 전사 후 홀로 생활하며, 2019년 6월 12일에 피고 C의 재종숙모와 D의 피고 C의 딸에게 매매계약서를 맺고 182만 원을 대금으로 매매를 완료했습니다. 이후, 피고 C은 2019년 10월 기억력 감퇴 증상으로 인해 I병원에 입원하여 인지기능 저하를 겪고 있었으며, 2020년 2월에는 신체감정에서 우울, 불안 등의 증상으로 어려움을 겪고 있었던 것으로 판단됩니다. 이러한 상황에서 변호사 B을 특별대리인으로 선임하여 원고의 권익을 보호하기 위한 조치를 취했습니다.',
   'role': 'user'},
  {'content': '관련 법조항 민사소송법 제62조 제2항 결론 피고 C은 원고에게, 가. 별지 목록 기재 부동산에 관하여 대구지방법원 등기국 2019. 6. 12. 접수 제84235호로 마친 소유권이전등기의 말소등기절차를 이행하라. 25,038,350원 및 이에 대하여 2019. 6. 27.부터 2021. 6. 22.까지는 연 5%, 그 다음날부터 완제일까지는 12%의 각 비율에 의한 돈을 지급하라. 원고의 피고 C에 대한 나머지 청구와 피고 D에 대한 청구를 각 기각한다. 소송비용 중 원고와 피고 C 사이에 생긴 부분의 95%는 위 피고가, 나머지는 원고가 각 부담하고, 원고와 피고 D 사이에 생긴 부분은 원고가 부담한다. 제1의 나.항은 가집행할 수 있다.',
   'role': 'assistant'}]}

In [ ]:
del_law_list(test_dataset[1])

{'messages': [{'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.',
   'role': 'system'},
  {'content': '사건유형 민사 세부유형 민사 1심 원고 측의 주장 없음 피고 측의 주장 없음 기초사실 요약 1935년생 원고는 전사 후 홀로 생활하며, 2019년 6월 12일에 피고 C의 재종숙모와 D의 피고 C의 딸에게 매매계약서를 맺고 182만 원을 대금으로 매매를 완료했습니다. 이후, 피고 C은 2019년 10월 기억력 감퇴 증상으로 인해 I병원에 입원하여 인지기능 저하를 겪고 있었으며, 2020년 2월에는 신체감정에서 우울, 불안 등의 증상으로 어려움을 겪고 있었던 것으로 판단됩니다. 이러한 상황에서 변호사 B을 특별대리인으로 선임하여 원고의 권익을 보호하기 위한 조치를 취했습니다.',
   'role': 'user'},
  {'content': "[' 피고 C은 원고에게, 가. 별지 목록 기재 부동산에 관하여 대구지방법원 등기국 2019. 6. 12. 접수 제84235호로 마친 소유권이전등기의 말소등기절차를 이행하라. 25,038,350원 및 이에 대하여 2019. 6. 27.부터 2021. 6. 22.까지는 연 5%, 그 다음날부터 완제일까지는 12%의 각 비율에 의한 돈을 지급하라. 원고의 피고 C에 대한 나머지 청구와 피고 D에 대한 청구를 각 기각한다. 소송비용 중 원고와 피고 C 사이에 생긴 부분의 95%는 위 피고가, 나머지는 원고가 각 부담하고, 원고와 피고 D 사이에 생긴 부분은 원고가 부담한다. 제1의 나.항은 가집행할 수 있다.']",
   'role': 'assistant'}]}

In [ ]:
new_train_dataset = train_dataset.map(del_law_list)
new_eval_dataset = eval_dataset.map(del_law_list)
new_test_dataset = test_dataset.map(del_law_list)

Map:   0%|          | 0/7801 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
new_test_dataset[1]

{'messages': [{'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.',
   'role': 'system'},
  {'content': '사건유형 민사 세부유형 민사 1심 원고 측의 주장 없음 피고 측의 주장 없음 기초사실 요약 1935년생 원고는 전사 후 홀로 생활하며, 2019년 6월 12일에 피고 C의 재종숙모와 D의 피고 C의 딸에게 매매계약서를 맺고 182만 원을 대금으로 매매를 완료했습니다. 이후, 피고 C은 2019년 10월 기억력 감퇴 증상으로 인해 I병원에 입원하여 인지기능 저하를 겪고 있었으며, 2020년 2월에는 신체감정에서 우울, 불안 등의 증상으로 어려움을 겪고 있었던 것으로 판단됩니다. 이러한 상황에서 변호사 B을 특별대리인으로 선임하여 원고의 권익을 보호하기 위한 조치를 취했습니다.',
   'role': 'user'},
  {'content': "[' 피고 C은 원고에게, 가. 별지 목록 기재 부동산에 관하여 대구지방법원 등기국 2019. 6. 12. 접수 제84235호로 마친 소유권이전등기의 말소등기절차를 이행하라. 25,038,350원 및 이에 대하여 2019. 6. 27.부터 2021. 6. 22.까지는 연 5%, 그 다음날부터 완제일까지는 12%의 각 비율에 의한 돈을 지급하라. 원고의 피고 C에 대한 나머지 청구와 피고 D에 대한 청구를 각 기각한다. 소송비용 중 원고와 피고 C 사이에 생긴 부분의 95%는 위 피고가, 나머지는 원고가 각 부담하고, 원고와 피고 D 사이에 생긴 부분은 원고가 부담한다. 제1의 나.항은 가집행할 수 있다.']",
   'role': 'assistant'}]}

In [ ]:
len(new_train_dataset)

7801

In [ ]:
type(new_train_dataset[0])

dict

In [ ]:
type(new_train_dataset)

datasets.arrow_dataset.Dataset

#SFTTrainer에 사용할 하이퍼파라미터 설정

In [ ]:
train_parameters = SFTConfig(
    output_dir=PROJECT_RUN_NAME,
    num_train_epochs=5,
    packing=False,

    per_device_train_batch_size=4,
    per_device_eval_batch_size=8,
    gradient_accumulation_steps=2,#너무 크게 설정하면 학습속도 느려지고, 학습 성능 안나오니 잘 조절하기

    gradient_checkpointing=True,
    optim="adamw_torch_fused",

    learning_rate=1e-4,
    fp16=True if torch_dtype == torch.float16 else False,
    bf16=True if torch_dtype == torch.bfloat16 else False,
    max_grad_norm=0.3,
    warmup_ratio=0.03,
    lr_scheduler_type="cosine",

    eval_strategy="epoch",
    save_strategy="epoch",
    save_total_limit=5,
    load_best_model_at_end=True,
    metric_for_best_model="eval_loss",
    greater_is_better=False,
    save_safetensors=True,

    logging_steps=50,
    report_to="wandb" if LOG_TO_WANDB else "tensorboard",
    run_name=RUN_NAME,

    push_to_hub=True,
    hub_strategy="every_save",
    hub_model_id=HUB_MODEL_NAME,
    hub_private_repo=True,

    dataset_kwargs={
        "add_special_tokens": False,
        "append_concat_token": True,
    }
)


trainer = SFTTrainer(
    model=base_model,
    args=train_parameters,
    train_dataset=new_train_dataset,
    eval_dataset=new_eval_dataset,
    peft_config=peft_config,
    processing_class=tokenizer
)

/usr/local/lib/python3.12/dist-packages/peft/mapping_func.py:72: UserWarning: You are trying to modify a model with PEFT for a second time. If you want to reload the model with a different config, make sure to call `.unload()` before.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(


Tokenizing train dataset:   0%|          | 0/7801 [00:00<?, ? examples/s]

Truncating train dataset:   0%|          | 0/7801 [00:00<?, ? examples/s]

Tokenizing eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

Truncating eval dataset:   0%|          | 0/100 [00:00<?, ? examples/s]

In [ ]:
#학습시작
trainer.train()

The tokenizer has new PAD/BOS/EOS tokens that differ from the model config and generation config. The model config and generation config were aligned accordingly, being updated with the tokenizer's values. Updated tokens: {'eos_token_id': 1, 'bos_token_id': 2, 'pad_token_id': 0}.
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,1.151900,1.126672,1.131242,4184740.000000,0.707956


wandb: Adding directory to artifact (law_ai5-2025-12-04_21.54.03/checkpoint-976)... Done. 1.4s
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)


Epoch,Training Loss,Validation Loss,Entropy,Num Tokens,Mean Token Accuracy
1,1.151900,1.126672,1.131242,4184740.000000,0.707956
2,1.074900,1.096053,1.052688,8369480.000000,0.712930
3,0.998600,1.089128,0.992046,12554220.000000,0.715597
4,0.925100,1.101534,0.929363,16738960.000000,0.716704
5,0.889500,1.119949,0.892995,20923700.000000,0.715203


wandb: Adding directory to artifact (law_ai5-2025-12-04_21.54.03/checkpoint-1952)... Done. 1.4s
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to preserve the current default behavior, you can pass use_reentrant=True. Refer to docs for more details on the differences between the two variants.
  return fn(*args, **kwargs)
wandb: Adding directory to artifact (law_ai5-2025-12-04_21.54.03/checkpoint-2928)... Done. 1.4s
/usr/local/lib/python3.12/dist-packages/torch/_dynamo/eval_frame.py:1044: UserWarning: torch.utils.checkpoint: the use_reentrant parameter should be passed explicitly. Starting in PyTorch 2.9, calling checkpoint without use_reentrant will raise an exception. use_reentrant=False is recommended, but if you need to pre

TrainOutput(global_step=4880, training_loss=1.042340256347031, metrics={'train_runtime': 9571.9396, 'train_samples_per_second': 4.075, 'train_steps_per_second': 0.51, 'total_flos': 6.263175000819469e+17, 'train_loss': 1.042340256347031, 'epoch': 5.0})

In [ ]:
#마지막 모델 저장
trainer.model.push_to_hub(PROJECT_RUN_NAME, private=True)
print(f"Saved to the hub: {PROJECT_RUN_NAME}")

README.md:   0%|          | 0.00/1.56k [00:00<?, ?B/s]

Processing Files (0 / 0)      : |          |  0.00B /  0.00B            

New Data Upload               : |          |  0.00B /  0.00B            

  ...adapter_model.safetensors:  16%|#5        | 41.9MB /  262MB            

No files have been modified since last commit. Skipping to prevent empty commit.


Saved to the hub: law_ai5-2025-12-04_21.54.03


In [ ]:
#wandb 종료
if LOG_TO_WANDB:
  wandb.finish()

eval/entropy,█▆▄▂▁
eval/loss,█▂▁▃▇
eval/mean_token_accuracy,▁▅▇█▇
eval/num_tokens,▁▃▅▆█
eval/runtime,█▃▁▁▁
eval/samples_per_second,▁▆███
eval/steps_per_second,▁▆███
train/entropy,█▄▄▄▃▃▃▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/epoch,▁▁▁▂▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▄▄▄▄▅▅▆▆▆▆▆▇▇▇▇█████
train/global_step,▁▁▁▂▂▃▃▃▃▃▄▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇▇▇▇█████
+5,...


In [ ]:
# free the memory again
del base_model
del trainer
torch.cuda.empty_cache()

#모델 출력 테스트

In [ ]:
RUN_NAME = "2025-12-04_21.54.03"
PROJECT_NAME = 'law_ai5'
PROJECT_RUN_NAME = f"{PROJECT_NAME}-{RUN_NAME}"
REVISION = "" # or REVISION = None
FINETUNED_MODEL = f"{HF_USER}/{PROJECT_RUN_NAME}"

In [ ]:
FINETUNED_MODEL

'dlddu123/law_ai5-2025-12-04_21.54.03'

In [ ]:
from peft import PeftModel

# Load Model base model
model = MODEL_CLASS.from_pretrained(MODEL_ID, low_cpu_mem_usage=True)

# Merge LoRA and base model and save
peft_model = PeftModel.from_pretrained(model,FINETUNED_MODEL)


processor = AutoTokenizer.from_pretrained(TOKENIZER_MODEL)
processor.save_pretrained("merged_model")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

adapter_config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

adapter_model.safetensors:   0%|          | 0.00/262M [00:00<?, ?B/s]

('merged_model/tokenizer_config.json',
 'merged_model/special_tokens_map.json',
 'merged_model/chat_template.jinja',
 'merged_model/tokenizer.model',
 'merged_model/added_tokens.json',
 'merged_model/tokenizer.json')

In [ ]:
new_test_dataset[13]

{'messages': [{'content': '원고 측의 주장과 피고 측의 주장을 살펴보았을때, 내릴 수 있는 판결을 예측하여 관련 법조항과 결론을 작성해줘.',
   'role': 'system'},
  {'content': '사건유형 민사 세부유형 민사 2심 원고 측의 주장 피고 2는 임대차계약을 갱신 중지 요청으로 인해 예금보험공사에 대항할 수 없게 되었고, 그 결과로 임대차계약 목적물인 아파트를 인도할 의무가 발생했습니다. 이는 계약 기간이 만료되어 피고 2가 채무를 이행할 수 없다는 것을 의미합니다. 피고 측의 주장 없음 기초사실 피고에게 제공된 아파트의 임대차계약 갱신 거절 및 임대차보증금 반환채권 압류 관련 문제들을 요약하면 다음과 같습니다. 핵심 내용 피고는 2017년 4월 21일에 지상 건물인 0000에 소유권 이전등기를 완료했지만, 임대차계약 갱신을 거절했습니다. 이러한 거절로 인해 피고 1은 임대차보증금 반환채권 압류 및 추심명령을 받았고, 이는 임대차계약 갱신 중지 요청에 대한 대응으로 보입니다. 상세 설명 임대차계약 갱신 거절 피고 1은 2017년 4월 21일에 지상 건물인 0000에 대한 임대차계약 갱신을 거절했습니다. 이는 임대차계약의 조건(예 특정 조건 하에 갱신 거절)에 따라 결정된 것으로 보입니다. 임대차보증금 반환채권 압류 및 추심 피고 1의 거절로 인해 피고 1은 임대차보증금 반환채권 압류 및 추심 명령을 받았습니다. 이는 피고 1이 임대차계약에 따라 임대보증금을 반환해야 하는 상황을 의미합니다. 계약 갱신 중지 요청 피고 1은 임대차계약 갱신을 중지하려는 요청을 보였습니다. 이는 피고 1이 임대차계약 조건에 따라 갱신을 거부한 상황을 반영합니다. 추가적인 고려 사항 임대차보증',
   'role': 'user'},
  {'content': "[' 그렇다면 원고의 피고 1 에 대한 청구는 위 인정범위 내에서 이유 있어 인용하고 나머지 청구는 이유 없어 기각하며, 피고 2 에 대한 청구는 이유 없어 기각하여야 한다. 제1심 

In [ ]:
tokenizer = processor

In [ ]:
from transformers import pipeline
from random import randint
import re

# Load the model and tokenizer into the pipeline
pipe = pipeline("text-generation", model=peft_model, tokenizer=tokenizer)

# Load a random sample from the test dataset
rand_idx = randint(0, len(test_dataset))
test_sample = new_test_dataset[19]

# Convert as test example into a prompt with the Gemma template
stop_token_ids = [tokenizer.eos_token_id, tokenizer.convert_tokens_to_ids("<end_of_turn>")]
prompt = pipe.tokenizer.apply_chat_template(test_sample["messages"][:2], tokenize=False, add_generation_prompt=True)

# Generate our SQL query.
outputs = pipe(prompt, max_new_tokens=300, do_sample=False, temperature=1, eos_token_id=stop_token_ids, disable_compile=True)

# Extract the user query and original answer

print(f"Original Answer:\n{test_sample['messages'][2]['content']}")
print(f"Generated Answer:\n{outputs[0]['generated_text'][len(prompt):].strip()}")

Device set to use cuda:0


Original Answer:
[' 그렇다면, 위 인정범위 내의 피고의 반소청구는 이유 있어 인용하고, 원고들의 본소청구와 피고의 나머지 반소청구는 모두 이유 없어 기각하여야 한다. 이 법원에서 추가한 청구를 포함하여 제1심판결(이 법원에서 소 취하로 실효된 부분 제외)을 다음과 같이 변경한다. 피고(반소원고)에게, 원고(반소피고) 1, 원고(반소피고) 2는 각 24,846,187원, 원고(반소피고) 3은 33,170,845원 및 각 이에 대하여 2018. 7. 26. 부터 2019. 11. 14. 까지 연 5%의, 그 다음날부터 다 갚는 날까지 연 15%의 각 비율로 계산한 돈을 지급하라. 원고(반소피고)들의 본소청구와 피고(반소원고)의 나머지 반소청구를 모두 기각한다. 소송 총비용은 본소와 반소를 통틀어 1/2은 원고(반소피고)들이, 나머지는 피고(반소원고)가 각 부담한다.']
Generated Answer:
[' 제1심 판결 중 아래에서 지급을 명하는 금원을 초과하는 피고 패소 부분을 취소하고, 그 취소 부분에 해당하는 원고들의 청구를 모두 기각한다. 피고는 원고 1에게 1,000만 원, 원고 2에게 1,000만 원, 원고 3에게 1,000만 원 및 위 각 금원에 대하여 2012. 1. 1.부터 2013. 12. 12.까지는 연 5%, 그 다음날부터 다 갚는 날까지는 연 20%의 각 비율에 의한 금원을 지급하라. 원고들의 항소 및 피고의 나머지 항소를 모두 기각한다. 소송총비용 중 1/2은 원고들이, 나머지는 피고가 각 부담한다.']
